# 추천 알고리즘 실습해보기, 코사인 유사도를 가지고!

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/data/소상공인시장진흥공단_상가(상권)정보_부산_20200630/부산_상권정보.csv', sep='|')
df.head(1)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,20031733,동해제일산오징어,NaN,Q,음식,Q03,일식/수산물,Q03A13,낙지/오징어,I56111,한식 음식점업,26,부산광역시,26260,동래구,2626057000,온천3동,2626010800,온천동,2626010800013800002,1,대지,1380,2.0,부산광역시 동래구 온천동 1380-2번지,262604190330,부산광역시 동래구 아시아드대로220번길,30,NaN,2626010800113800002003460,NaN,"부산광역시 동래구 아시아드대로220번길 30, (온천동)",607060,47838.0,NaN,1,NaN,129.068324,35.202902


In [ ]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [ ]:
df['행정동명']

0         온천3동
1          괘법동
2         범천1동
3         명지1동
4         범천1동
          ... 
154449    봉래1동
154450     녹산동
154451     중앙동
154452    장전2동
154453    하단1동
Name: 행정동명, Length: 154454, dtype: object

In [ ]:
# 음식점 데이터만 이용
df = df.loc[df['상권업종대분류명'] == '음식']

# 필요한 칼럼만을 뽑아냅니다.
df = df[['상호명','상권업종중분류명','상권업종소분류명','표준산업분류명','행정동명','위도','경도']]

# 그 중에서, 사용할 동을 뽑아냅니다.
df = df.loc[(df['행정동명'] == '부전2동')]
df = df.dropna()

In [ ]:
df.head(5)

,상호명,상권업종중분류명,상권업종소분류명,표준산업분류명,행정동명,위도,경도
240,오스카Ⅰ,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.152243,129.057125
268,부산양곱창1호,한식,곱창/양구이전문,한식 음식점업,부전2동,35.154759,129.057004
569,칠공공비어,한식,한식/백반/한정식,한식 음식점업,부전2동,35.153555,129.058591
664,아웃백스테이크하우스부전점,양식,패밀리레스토랑,서양식 음식점업,부전2동,35.154226,129.060158
683,부산항가라오케,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.156700,129.054581


In [ ]:
# 컬럼명 단순화
# 순서대로 상호명, 상권업종중분류명, 상권업종소분류명, 표준산업분류명, 행정동명, 위도, 경도 입니다.
df.columns = ['name','cat_mid','cat_small','cat_std','dong','lon','lat']

In [ ]:
df['cat_mix'] = df['cat_mid'] + df['cat_small'] + df['cat_std']
df['cat_mix'] = df['cat_mix'].str.replace("/"," ")

df.head(5)

,name,cat_mid,cat_small,cat_std,dong,lon,lat,cat_mix
240,오스카Ⅰ,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.152243,129.057125,유흥주점룸살롱 단란주점일반유흥 주점업
268,부산양곱창1호,한식,곱창/양구이전문,한식 음식점업,부전2동,35.154759,129.057004,한식곱창 양구이전문한식 음식점업
569,칠공공비어,한식,한식/백반/한정식,한식 음식점업,부전2동,35.153555,129.058591,한식한식 백반 한정식한식 음식점업
664,아웃백스테이크하우스부전점,양식,패밀리레스토랑,서양식 음식점업,부전2동,35.154226,129.060158,양식패밀리레스토랑서양식 음식점업
683,부산항가라오케,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.156700,129.054581,유흥주점룸살롱 단란주점일반유흥 주점업


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  # 피쳐 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cat_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [ ]:
place_simi_cate

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.16903085, ..., 0.16903085, 0.        ,
        0.16903085],
       [0.        , 0.16903085, 1.        , ..., 1.        , 0.        ,
        0.14285714],
       ...,
       [0.        , 0.16903085, 1.        , ..., 1.        , 0.        ,
        0.14285714],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.16903085, 0.14285714, ..., 0.14285714, 0.        ,
        1.        ]])

In [ ]:
# install chromium, its driver, and selenium
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (84.0.4147.105-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
driver = webdriver.Chrome('chromedriver',options=options)

1. 공공데이터로 확보한 상호명과 행정동명을 검색어로 변환하여 포털에 검색합니다.
2. 검색 결과로 나온 블로그 리뷰 및 별점 데이터를 가져옵니다.

In [ ]:
df['kakao_keyword'] = df['dong'] + "%20" + df['name']
df['kakao_map_url'] = ''

print("여기서 %20은 띄어쓰기를 의미해요")
for i, keyword in enumerate(df['kakao_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0]} 행", keyword)
    try:
        kakao_map_search_url = f"https://map.kakao.com/?q={keyword}"
        driver.get(kakao_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # child(1)이 없던데요?
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.moreview").get_attribute('href')
                time.sleep(1)
            except Exception as e2:
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass

In [ ]:
df.head(5)

,name,cat_mid,cat_small,cat_std,dong,lon,lat,cat_mix,kakao_keyword,kakao_map_url
240,오스카Ⅰ,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.152243,129.057125,유흥주점룸살롱 단란주점일반유흥 주점업,부전2동%20오스카Ⅰ,NaN
268,부산양곱창1호,한식,곱창/양구이전문,한식 음식점업,부전2동,35.154759,129.057004,한식곱창 양구이전문한식 음식점업,부전2동%20부산양곱창1호,https://place.map.kakao.com/15563128
569,칠공공비어,한식,한식/백반/한정식,한식 음식점업,부전2동,35.153555,129.058591,한식한식 백반 한정식한식 음식점업,부전2동%20칠공공비어,https://place.map.kakao.com/9902660
664,아웃백스테이크하우스부전점,양식,패밀리레스토랑,서양식 음식점업,부전2동,35.154226,129.060158,양식패밀리레스토랑서양식 음식점업,부전2동%20아웃백스테이크하우스부전점,NaN
683,부산항가라오케,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.156700,129.054581,유흥주점룸살롱 단란주점일반유흥 주점업,부전2동%20부산항가라오케,https://place.map.kakao.com/21423596


In [ ]:
df.isnull().sum() 

#map_url에서 500개의 null값이 보입니다. 
#주소가 검색되지 않는 곳은 없다고 생각하고, drop하도록 합니다.

name               0
cat_mid            0
cat_small          0
cat_std            0
dong               0
lon                0
lat                0
cat_mix            0
kakao_keyword      0
kakao_map_url    500
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

kakao_map_name_list = []
blog_review_list = []
blog_review_qty_list = []
kakao_map_star_review_stars_list = []
kakao_map_star_review_qty_list = []

for i, url in enumerate(tqdm_notebook(df['kakao_map_url'])):
    driver.get(url)
    time.sleep(1)
    review_text = ""
    
    try:
        kakao_map_name = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > h2").text
        blog_review_qty = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(5) > span").text
        star_review_stars = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b.inactive").text
        star_review_qty = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g").text
    except Exception as e1:
        if "inactive" in str(e1):
            star_review_stars = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b").text
            star_review_qty = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g").text
            
    num = 0
    try:
        while num < 3:
            try:
                num += 1
                review_text = review_text + driver.find_element_by_css_selector(f"div.wrap_list > ul > li:nth-child({num}) > a > div.review_story > p").text
                
                if num == 3:
                    blog_review_list.append(review_text)  # 3개나 찾았으면 그만 찾고 나감
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    break

            except Exception as e1:
                if "li:nth-child(1)" in str(e1):  # child(1)이 없던데요? -> 리뷰가 하나도 없는 것
                    review_text = "empty"
                    blog_review_list.append(review_text)
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    break
                else:
                    blog_review_list.append(review_text)  # 일단 리뷰가 하나도 없는 건 아니니 붙이고 탈출 / 리뷰 딸랑 하나 있으면 발생할 수 있음                    
                    kakao_map_name_list.append(kakao_map_name)
                    blog_review_qty_list.append(blog_review_qty)
                    kakao_map_star_review_stars_list.append(star_review_stars)
                    kakao_map_star_review_qty_list.append(star_review_qty)
                    break
                
    except Exception as e2:
        print(e2)
driver.quit()


df['kakao_store_name'] = kakao_map_name_list  # 카카오 상세페이지에서 크롤링한 상호명
df['kakao_star_point'] = kakao_map_star_review_stars_list  # 카카오 상세페이지에서 평가한 별점 평점
df['kakao_star_point_qty'] = kakao_map_star_review_qty_list  # 카카오 상세페이지에서 별점 평가를 한 횟수
df['kakao_blog_review_txt'] = blog_review_list  # 카카오 상세페이지에 나온 블로그 리뷰 텍스트들
df['kakao_blog_review_qty'] = blog_review_qty_list  # 카카오 상세페이지에 나온 블로그 리뷰의 총 개수

In [ ]:
df.isnull().sum()

Unnamed: 0               0
name                     0
cat_mid                  0
cat_small                0
cat_std                  0
dong                     0
lon                      0
lat                      0
cat_mix                  0
kakao_keyword            0
kakao_map_url            0
kakao_store_name         0
kakao_star_point         0
kakao_star_point_qty     0
kakao_blog_review_txt    0
kakao_blog_review_qty    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             885 non-null    int64  
 1   name                   885 non-null    object 
 2   cat_mid                885 non-null    object 
 3   cat_small              885 non-null    object 
 4   cat_std                885 non-null    object 
 5   dong                   885 non-null    object 
 6   lon                    885 non-null    float64
 7   lat                    885 non-null    float64
 8   cat_mix                885 non-null    object 
 9   kakao_keyword          885 non-null    object 
 10  kakao_map_url          885 non-null    object 
 11  kakao_store_name       885 non-null    object 
 12  kakao_star_point       885 non-null    object 
 13  kakao_star_point_qty   885 non-null    object 
 14  kakao_blog_review_txt  885 non-null    object 
 15  kakao_

* 카테고리가 얼마나 유사한지
* 블로그 리뷰가 얼마나 유사한지
* 블로그 리뷰가 얼마나 많이 올라왔는지
* 블로그 별점이 얼마나 높은지
* 블로그 별점 평가가 얼마나 많이 됐는지
위 공식 5개를 적절한 비율로 가중치를 주어 공식을 만들어봅니다.

In [ ]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['kakao_blog_review_txt']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [ ]:
df.head(5)

,Unnamed: 0,name,cat_mid,cat_small,cat_std,dong,lon,lat,cat_mix,kakao_keyword,kakao_map_url,kakao_store_name,kakao_star_point,kakao_star_point_qty,kakao_blog_review_txt,kakao_blog_review_qty
0,268,부산양곱창1호,한식,곱창/양구이전문,한식 음식점업,부전2동,35.154759,129.057004,한식곱창 양구이전문한식 음식점업,부전2동%20부산양곱창1호,https://place.map.kakao.com/15563128,친구양곱창 문현1호점,4.4,(6),친구양곱창 문현동 매일 ㅣ 15:00 - 24:00 *일요일 휴무 전화 ㅣ051-6...,3
1,569,칠공공비어,한식,한식/백반/한정식,한식 음식점업,부전2동,35.153555,129.058591,한식한식 백반 한정식한식 음식점업,부전2동%20칠공공비어,https://place.map.kakao.com/9902660,700비어 서면일번가점,0,(0),요즘 날씨가 장마가 오니마니 해서 너무 찝찝하다 더운 여름이 오고 있음을 느낀다 찝...,2
2,683,부산항가라오케,유흥주점,룸살롱/단란주점,일반유흥 주점업,부전2동,35.156700,129.054581,유흥주점룸살롱 단란주점일반유흥 주점업,부전2동%20부산항가라오케,https://place.map.kakao.com/21423596,부산항가라오케,0,(0),empty,0
3,747,베르빌골프아카데미,한식,한식/백반/한정식,한식 음식점업,부전2동,35.153880,129.055401,한식한식 백반 한정식한식 음식점업,부전2동%20베르빌골프아카데미,https://place.map.kakao.com/17036813,베르빌 스크린골프아카데미,5.0,(1),empty,0
4,751,대림참치,한식,한식/백반/한정식,한식 음식점업,부전2동,35.154328,129.056812,한식한식 백반 한정식한식 음식점업,부전2동%20대림참치,https://place.map.kakao.com/16243380,대림참치 서현점,3.7,(7),#서면참치맛집 #서면참치 히힛 어제 저녁은 기름지고 땟깔좋은 Rich Rich한 참...,21


In [ ]:
for i in range(len(df)):
  df['kakao_star_point_qty'][i] = int(df['kakao_star_point_qty'][i][1])

In [ ]:
df = df.astype({'kakao_blog_review_qty': np.float, 
                'kakao_star_point': np.float,
                'kakao_star_point_qty':np.float})

In [ ]:
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.

place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                 + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                 + np.repeat([df['kakao_blog_review_qty'].values], len(df['kakao_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 + np.repeat([df['kakao_star_point'].values], len(df['kakao_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 + np.repeat([df['kakao_star_point_qty'].values], len(df['kakao_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )



# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

송정 3대국밥을 포함한 5개의 업체를 뽑아봅니다.

송정3대국밥과 가장 유사한 업체는 송정국밥, 경주국밥, 홍유단, 꾸이쩜빵이 나왔네요.

In [ ]:
find_simi_place(df, place_simi_co_sorted_ind, '송정3대국밥', 5)

,name,cat_mid,cat_small,cat_std,dong,lon,lat,cat_mix,kakao_keyword,kakao_map_url,kakao_store_name,kakao_star_point,kakao_star_point_qty,kakao_blog_review_txt,kakao_blog_review_qty
408,송정3대국밥,한식,해장국/감자탕,한식 음식점업,부전2동,35.155674,129.058541,한식해장국 감자탕한식 음식점업,부전2동%20송정3대국밥,https://place.map.kakao.com/10551889,송정삼대국밥,3.5,1.0,제목 :┃부산진구.부전동┃송정3대국밥 (서면 돼지국밥) 상호 : 송정3대국밥 주소 ...,416.0
398,송정국밥,한식,한식/백반/한정식,한식 음식점업,부전2동,35.155674,129.058541,한식한식 백반 한정식한식 음식점업,부전2동%20송정국밥,https://place.map.kakao.com/10551889,송정삼대국밥,3.5,1.0,제목 :┃부산진구.부전동┃송정3대국밥 (서면 돼지국밥) 상호 : 송정3대국밥 주소 ...,416.0
455,경주국밥,한식,해장국/감자탕,한식 음식점업,부전2동,35.155674,129.058541,한식해장국 감자탕한식 음식점업,부전2동%20경주국밥,https://place.map.kakao.com/25619797,경주박가국밥 서면점,4.2,1.0,안녕하세요 콩순이입니다~ 부산 여행 가면 꼭 먹게되는 음식 돼지국밥! 이번엔 서면시...,37.0
347,홍유단,중식,중국음식/중국집,중식 음식점업,부전2동,35.153267,129.056513,중식중국음식 중국집중식 음식점업,부전2동%20홍유단,https://place.map.kakao.com/25976846,홍유단,3.8,2.0,안녕하세요 여러분 ! 유리얼입니다 ? 오늘은 서면에서 굉장히 크고 유명한 서면 중식...,433.0
645,꾸이점빵,한식,기타고기요리,한식 음식점업,부전2동,35.153681,129.060732,한식기타고기요리한식 음식점업,부전2동%20꾸이점빵,https://place.map.kakao.com/107987366,꾸이점빵,3.0,2.0,서면 고기집 추천 꾸이점빵 #눈꽃살모듬 안녕하세요. 구지입니다 ㅎㅎ 오랜만에 친한동...,392.0


In [ ]:
find_simi_place(df, place_simi_co_sorted_ind, '가야밀면', 5)

,name,cat_mid,cat_small,cat_std,dong,lon,lat,cat_mix,kakao_keyword,kakao_map_url,kakao_store_name,kakao_star_point,kakao_star_point_qty,kakao_blog_review_txt,kakao_blog_review_qty
81,가야밀면,분식,국수/만두/칼국수,분식 및 김밥 전문점,부전2동,35.155154,129.061267,분식국수 만두 칼국수분식 및 김밥 전문점,부전2동%20가야밀면,https://place.map.kakao.com/16668945,가야밀면,0.0,0.0,안녕하세요~!찌니에용 찌니는 부산놀러가면 서면을 주변으로 다녀요! 놀다가 출출할때 ...,3.0
143,국수가게,분식,국수/만두/칼국수,분식 및 김밥 전문점,부전2동,35.155172,129.057141,분식국수 만두 칼국수분식 및 김밥 전문점,부전2동%20국수가게,https://place.map.kakao.com/11495457,기장손칼국수,3.8,1.0,서면시장 오래된 맛집 기장손칼국수 기장손칼국수 부산광역시 부산진구 서면로 56 ㅎㅎ...,329.0
574,기장손칼국수,분식,국수/만두/칼국수,분식 및 김밥 전문점,부전2동,35.155677,129.058168,분식국수 만두 칼국수분식 및 김밥 전문점,부전2동%20기장손칼국수,https://place.map.kakao.com/11495457,기장손칼국수,3.8,1.0,서면시장 오래된 맛집 기장손칼국수 기장손칼국수 부산광역시 부산진구 서면로 56 ㅎㅎ...,329.0
102,칼국수집,분식,국수/만두/칼국수,분식 및 김밥 전문점,부전2동,35.155677,129.058168,분식국수 만두 칼국수분식 및 김밥 전문점,부전2동%20칼국수집,https://place.map.kakao.com/11495457,기장손칼국수,3.8,1.0,서면시장 오래된 맛집 기장손칼국수 기장손칼국수 부산광역시 부산진구 서면로 56 ㅎㅎ...,329.0
530,고향손칼국수,분식,국수/만두/칼국수,분식 및 김밥 전문점,부전2동,35.155677,129.058168,분식국수 만두 칼국수분식 및 김밥 전문점,부전2동%20고향손칼국수,https://place.map.kakao.com/27503713,고향손칼국수,3.4,5.0,고향칼국수 부산광역시 부산진구 부전동 256-6 서면시장 내 고향칼국수 051-81...,15.0
